# {glue:text}`jupyter-book_github_org`

**Activity from {glue:}`jupyter-book_start` to {glue:}`jupyter-book_stop`**

In [1]:
from datetime import date
from dateutil.relativedelta import relativedelta
from myst_nb import glue
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt
import os.path as op
from json import loads

from warnings import simplefilter
simplefilter('ignore')
pd.set_option('display.max_columns', None) # 'None' means show all

In [2]:
# Altair config
def author_url(author):
    return f"https://github.com/{author}"

def alt_theme():
    return {
        'config': {
            'axisLeft': {
                'labelFontSize': 15,
            },
            'axisBottom': {
                'labelFontSize': 15,
            },
        }
    }

alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")


# Define colors we'll use for GitHub membership
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_palette = np.array(sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)) * 256
author_colors = ["rgb({}, {}, {})".format(*color) for color in author_palette]
author_color_dict = {key: val for key, val in zip(author_types, author_palette)}

In [3]:
github_org = "jupyterhub"
top_n_repos = 15
n_days = 90

In [4]:
# Parameters
github_org = "jupyter-book"
n_days = 90


In [5]:
############################################################
# Variables
stop = date.today()
start = date.today() - relativedelta(days=n_days)

# Strings for use in queries
start_date = f"{start:%Y-%m-%d}"
stop_date = f"{stop:%Y-%m-%d}"

# Glue variables for use in markdown
glue(f"{github_org}_github_org", github_org, display=False)
glue(f"{github_org}_start", start_date, display=False)
glue(f"{github_org}_stop", stop_date, display=False)

## Load data

Load and clean up the data

In [6]:
from pathlib import Path
import sqlite3
path_data = Path("../data")
path_data = list(path_data.rglob(f"*{github_org}.db"))[0]

def df_from_sql(query, db):
    con = sqlite3.connect(db)
    return pd.read_sql(query, con)
    con.close()

In [7]:
print(f"Loading from {path_data}")
repos = df_from_sql("SELECT * FROM repos", path_data)
users = df_from_sql("SELECT * FROM users", path_data)
issues = df_from_sql("SELECT * FROM issues", path_data)
comments = df_from_sql("SELECT * FROM issue_comments", path_data)

# Merge repository information into issues
def merge_repo_into_df(df, kind):
    df = pd.merge(df, users[["id", "login"]], left_on="user", right_on="id", suffixes=("_issue", "_user"), how="inner")
    df = pd.merge(df, repos[["id", "full_name", "name"]], left_on="repo", right_on="id", suffixes=("_issue", "_repo"), how="inner")
    df["org"] = df["full_name"].map(lambda a: a.split('/')[0])
    df["repo"] = df["full_name"].map(lambda a: a.split('/')[1])
    df["url"] = "https://github.com/" + df["org"] + "/" + df["repo"] + f"/{kind}/" + df["number"].astype(str)
    return df
issues = merge_repo_into_df(issues, kind="issues")

# Pull out the Pull Requests from Issues
prs = issues.loc[~issues["pull_request"].isna()]
def identify_merger(closedby):
    if not closedby:
        return None
    try:
        cb = loads(closedby)
    except:
        print(closedby)
    return cb["login"]
prs["merged_by"] = prs["closed_by"].map(identify_merger)

# These issues now _remove_ the PRs
issues = issues.loc[issues["pull_request"].isna()]

# Add reactions data
def count_positive(a):
    a = loads(a)
    return a["+1"] + a["hooray"] + a["heart"]
issues["positive"] = issues["reactions"].map(count_positive)

# Drop comments if there is no associated issue
comments = comments.dropna(subset=["issue"])
comments["issue"] = comments.loc[:, "issue"].astype(int)

# Add repository information to comments
comments = pd.merge(comments, issues[["id_issue", "full_name", "org", "repo"]], left_on="issue", right_on="id_issue", how="inner", suffixes=("_comment", "_issue"))

# Merge user data into comments
comments = pd.merge(comments, users[["id", "login"]], left_on="user", right_on="id", suffixes=("_comment", "_user"), how="inner")

Loading from ../data/612f67cbff44837826a10e815fa2f115-jupyter-book.db


In [8]:
# Convert columns to datetime
datetime_columns = ["created_at", "closed_at"]
for df in [comments, issues, prs]:
    for col in datetime_columns:
        if col in df:
            df[col] = pd.to_datetime(df[col])
            # Strip timezone information so we can run simpler queries
            df[col] = df[col].dt.tz_localize(None)

In [9]:
# What are the top N repos, we will only plot these in the full data plots
top_commented_repos = comments.groupby("repo").count().sort_values("created_at", ascending=False)['created_at']
use_repos = top_commented_repos.head(top_n_repos).index.tolist()

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [10]:
merged = prs.query('state == "closed" and closed_at > @start_date and closed_at < @stop_date')

In [11]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['login'].reset_index().sort_values(['org', 'login'], ascending=False)
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='login',
    color='org'
)

alt.Chart(...)

### Authoring and merging stats by repository

Let's see who has been doing most of the PR authoring and merging. The PR author is generally the
person that implemented a change in the repository (code, documentation, etc). The PR merger is
the person that "pressed the green button" and got the change into the main codebase.

In [12]:
# Prep our merging DF
merged_by_repo = merged.groupby(['repo', 'login'], as_index=False).agg({'id': 'count', 'author_association': 'first'}).rename(columns={'id': "authored", 'login': 'username'})
closed_by_repo = merged.groupby(['name', 'merged_by']).count()['id_issue'].reset_index().rename(columns={'id_issue': "closed", "merged_by": "username"})

In [13]:
charts = []
title = f"PR authors for {github_org} in the last {n_days} days"
this_data = merged_by_repo.replace(np.nan, 0).groupby('username', as_index=False).agg({'authored': 'sum', 'author_association': 'first'})
this_data = this_data.sort_values('authored', ascending=False)
ch = alt.Chart(data=this_data, title=title).mark_bar().encode(
    x='username',
    y='authored',
    color=alt.Color('author_association', scale=alt.Scale(domain=author_types, range=author_colors))
)
ch

alt.Chart(...)

In [14]:
charts = []
title = f"Users that have merged PRs for {github_org} in the last {n_days} days"
ch = alt.Chart(data=closed_by_repo.replace(np.nan, 0), title=title).mark_bar().encode(
    x='username',
    y='closed',
)
ch

alt.Chart(...)

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [15]:
created = issues.query('state == "open" and created_at > @start_date and created_at < @stop_date')
closed = issues.query('state == "closed" and closed_at > @start_date and closed_at < @stop_date')

In [16]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
)

alt.Chart(...)

In [17]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
)

alt.Chart(...)

In [18]:
created_closed = pd.merge(created_counts.rename(columns={'number': 'created'}).drop(columns='org/repo'),
                          closed_counts.rename(columns={'number': 'closed'}).drop(columns='org/repo'),
                          on=['org', 'repo'], how='outer')

created_closed = pd.melt(created_closed, id_vars=['org', 'repo'], var_name="kind", value_name="count").replace(np.nan, 0)

In [19]:
charts = []
# Pick the top 10 repositories
top_repos = created_closed.groupby(['repo']).sum().sort_values(by='count', ascending=False).head(10).index
ch = alt.Chart(created_closed.query('repo in @top_repos'), width=120).mark_bar().encode(
    x=alt.X("kind", axis=alt.Axis(labelFontSize=15, title="")), 
    y=alt.Y('count', axis=alt.Axis(titleFontSize=15, labelFontSize=12)),
    color='kind',
    column=alt.Column("repo", header=alt.Header(title=f"Issue activity, last {n_days} days for {github_org}", titleFontSize=15, labelFontSize=12))
)
ch

alt.Chart(...)

In [20]:
# Set to datetime
for kind in ['created_at', 'closed_at']:
    closed.loc[:, kind] = pd.to_datetime(closed[kind])
    
closed.loc[:, 'time_open'] = closed['closed_at'] - closed['created_at']
closed.loc[:, 'time_open'] = closed['time_open'].dt.total_seconds()

In [21]:
time_open = closed.groupby(['org', 'repo']).agg({'time_open': 'median'}).reset_index()
time_open['time_open'] = time_open['time_open'] / (60 * 60 * 24)
time_open['org/repo'] = time_open.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = time_open.sort_values(['org', 'time_open'], ascending=False)['repo'].values
alt.Chart(data=time_open, title=f"Time to close for issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y=alt.Y('time_open', title="Median Days Open"),
)

alt.Chart(...)

## Most-upvoted issues

In [22]:
thumbsup = issues.query("state == 'open'").sort_values("positive", ascending=False).head(25)
thumbsup = thumbsup[["title", "url", "number", "positive", "repo"]]

text = []
for ii, irow in thumbsup.iterrows():
    itext = f"- ({irow['positive']}) {irow['title']} - {irow['repo']} - [#{irow['number']}]({irow['url']})"
    text.append(itext)
text = '\n'.join(text)
HTML(markdown(text))

HTML(value='<ul>\n<li>(35) Support for RISE slides - jupyter-book - <a href="https://github.com/jupyter-book/j…

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [23]:
commentors = (
    comments
    .query("created_at > @start_date and created_at < @stop_date")
    .groupby(['org', 'repo', 'login', 'author_association'])
    .count().rename(columns={'id_issue': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [24]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    idata = idata.groupby('login', as_index=False).agg({'count': 'sum', 'author_association': 'first'})
    idata = idata.sort_values('count', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), title=title).mark_bar().encode(
        x='login',
        y='count',
        color=alt.Color('author_association', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

alt.HConcatChart(...)

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [25]:
first_comments = []
latest_comments = comments.query("created_at > @start_date")
for (org, repo, issue_id), i_comments in latest_comments.groupby(['org', 'repo', 'id_issue']):
    ix_min = pd.to_datetime(i_comments['created_at']).idxmin()
    first_comment = i_comments.loc[ix_min]
    if isinstance(first_comment, pd.DataFrame):
        first_comment = first_comment.iloc[0]
    first_comments.append(first_comment)
first_comments = pd.concat(first_comments, axis=1).T

# Make up counts for viz
first_responder_counts = first_comments.groupby(['org', 'login', 'author_association'], as_index=False).\
    count().rename(columns={'id_issue': 'n_first_responses'}).sort_values(['org', 'n_first_responses'], ascending=False)


In [26]:
n_plot = 50

title = f"Top {n_plot} first responders for {github_org} in the last {n_days} days"
idata = first_responder_counts.groupby('login', as_index=False).agg({'n_first_responses': 'sum', 'author_association': 'first'})
idata = idata.sort_values('n_first_responses', ascending=False).head(n_plot)
ch = alt.Chart(data=idata.head(n_plot), title=title).mark_bar().encode(
    x='login',
    y='n_first_responses',
    color=alt.Color('author_association', scale=alt.Scale(domain=author_types, range=author_colors))
)
ch

alt.Chart(...)

## Recent activity

### A list of merged PRs by project

Below is a tabbed readout of recently-merged PRs. Check out the title to get an idea for what they
implemented, and be sure to thank the PR author for their hard work!

In [27]:
tabs = widgets.Tab(children=[])

for ii, ((org, repo), imerged) in enumerate(merged.query("repo in @use_repos").groupby(['org', 'repo'])):
    merged_by = {}
    pr_by = {}
    issue_md = []
    issue_md.append(f"#### Closed PRs for repo: [{org}/{repo}](https://github.com/{github_org}/{repo})")
    issue_md.append("")
    issue_md.append(f"##### ")

    for _, ipr in imerged.iterrows():
        user_name = ipr['login']
        user_url = author_url(user_name)
        pr_number = ipr['number']
        pr_html = ipr['url']
        pr_title = ipr['title']
        pr_closedby = ipr['merged_by']
        pr_closedby_url = f"https://github.com/{pr_closedby}"
        if user_name not in pr_by:
            pr_by[user_name] = 1
        else:
            pr_by[user_name] += 1

        if pr_closedby not in merged_by:
            merged_by[pr_closedby] = 1
        else:
            merged_by[pr_closedby] += 1
        text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
        issue_md.append(text)
    
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))

    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, repo)
tabs

### A list of recent issues

Below is a list of issues with recent activity in each repository. If they seem of interest
to you, click on their links and jump in to participate!

In [28]:
# Add comment count data to issues and PRs
comment_counts = (
    comments
    .query("created_at > @start_date and created_at < @stop_date")
    .groupby(['org', 'repo', 'id_issue'])
    .count().iloc[:, 0].to_frame()
)
comment_counts.columns = ['n_comments']
comment_counts = comment_counts.reset_index()

In [29]:
n_plot = 5
tabs = widgets.Tab(children=[])

for ii, (repo, i_issues) in enumerate(comment_counts.query("repo in @use_repos").groupby('repo')):
    
    issue_md = []
    issue_md.append("")
    issue_md.append(f"##### [{github_org}/{repo}](https://github.com/{github_org}/{repo})")

    top_issues = i_issues.sort_values('n_comments', ascending=False).head(n_plot)
    top_issue_list = pd.merge(issues, top_issues, left_on=['org', 'repo', 'id_issue'], right_on=['org', 'repo', 'id_issue'])
    for _, issue in top_issue_list.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
        user_name = issue['login']
        user_url = author_url(user_name)
        issue_number = issue['number']
        issue_html = issue['url']
        issue_title = issue['title']

        text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
        issue_md.append(text)

    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))

    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, repo)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 90 days